In [6]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
%%capture
# Install the latest Tensorflow version.
!pip3 install --upgrade tensorflow-gpu
# Install TF-Hub.
!pip3 install tensorflow-hub

In [0]:
from pathlib import Path
import re
from matplotlib import pyplot as plt
from tqdm import tqdm
import pandas as pd
import numpy as np
import pickle
import seaborn as sns
sns.set()

import tensorflow as tf
import tensorflow_hub as hub
# Reduce logging output
from absl import logging
logging.set_verbosity(logging.ERROR)

In [0]:
# Inference model
MODULE_URL = 'https://tfhub.dev/google/universal-sentence-encoder/4'
# Inference batch size
MAX_BATCH_SIZE = 1024

In [0]:
MAIN_DIR = Path('drive/My Drive/Colab Notebooks/transformers/MoviePlots')
DATA_FILE = MAIN_DIR/'data'/'data.pkl'

MODEL_DIR = MAIN_DIR/'text_generation'/'DistilGPT-2'
DUMPS_DIR = MODEL_DIR/'dumps'
CACHE_DIR = MODEL_DIR/'eval_cache'
CACHE_DIR.mkdir(exist_ok=True)

## Import training data

In [0]:
with open(DATA_FILE, 'rb') as handle:
    train_data = pickle.load(handle)

In [12]:
train_df = pd.DataFrame(train_data)

print(train_df.head())

                                               title  ...                      secondary
0                               "#7DaysLater" (2013)  ...                           None
1       "#BlackLove" (2015) {Crash the Party (#1.9)}  ...                Crash the Party
2  "#BlackLove" (2015) {Making Lemonade Out of Le...  ...  Making Lemonade Out of Lemons
3      "#BlackLove" (2015) {Miss Independent (#1.5)}  ...               Miss Independent
4     "#BlackLove" (2015) {Sealing the Deal (#1.10)}  ...               Sealing the Deal

[5 rows x 7 columns]


## Parse dumps

In [13]:
dump_files = list(DUMPS_DIR.glob('*.txt'))

dump_files

[PosixPath('drive/My Drive/Colab Notebooks/transformers/MoviePlots/text_generation/DistilGPT-2/dumps/temp_1_topp_0.txt'),
 PosixPath('drive/My Drive/Colab Notebooks/transformers/MoviePlots/text_generation/DistilGPT-2/dumps/temp_0_7_topp_0.txt'),
 PosixPath('drive/My Drive/Colab Notebooks/transformers/MoviePlots/text_generation/DistilGPT-2/dumps/temp_0_7_topp_0_9.txt'),
 PosixPath('drive/My Drive/Colab Notebooks/transformers/MoviePlots/text_generation/DistilGPT-2/dumps/temp_1_topp_0_9.txt'),
 PosixPath('drive/My Drive/Colab Notebooks/transformers/MoviePlots/text_generation/DistilGPT-2/dumps/temp_1_3_topp_0.txt'),
 PosixPath('drive/My Drive/Colab Notebooks/transformers/MoviePlots/text_generation/DistilGPT-2/dumps/temp_1_3_topp_0_9.txt')]

In [0]:
BOS_TOKEN = '<|startoftext|>'
EOG_TOKEN = '<|endofgenre|>'
EOS_TOKEN = '<|endoftext|>'

In [0]:
def parse_sample(line):
    """Extract the data from a line generated by the model."""
    main = line.split(BOS_TOKEN)[1].split(EOS_TOKEN)[0]
    genre = main.split(EOG_TOKEN)[0].strip()
    plot = main.split(EOG_TOKEN)[1].strip()
    return {
        'genre': genre,
        'plot': plot
    }

In [0]:
def parse_fname(fname):
    """Parse the name of a file for temperature and top_p."""
    pattern = r'temp_(.+)_topp_(.+).txt'
    match = re.search(pattern, fname)
    temp, topp = match.groups()
    temp = float(temp.replace('_', '.'))
    topp = float(topp)
    return temp, topp

In [0]:
def parse_dumps(dump_files, overwrite=False):
    """Read dumps and parse them."""
    fpath = CACHE_DIR/'data.pkl'
    if fpath.exists() and not overwrite:
        print("Loading from cache...")
        with open(fpath, 'rb') as handle:
            return pd.DataFrame(pickle.load(handle))
    gen_data = []
    for dump_file in dump_files:
        with dump_file.open() as f:
            for line in f:
                try:
                    data_dict = parse_sample(line)
                    temp, topp = parse_fname(dump_file.name)
                    data_dict['temp'] = temp
                    data_dict['topp'] = topp
                    gen_data.append(data_dict)
                except:
                    pass
    with open(fpath, 'wb') as handle:
        pickle.dump(gen_data, handle, protocol=pickle.HIGHEST_PROTOCOL)
    gen_df = pd.DataFrame(gen_data)
    return gen_df

In [18]:
gen_df = parse_dumps(dump_files, overwrite=True)

print(gen_df.shape)
print(gen_df.head())

(1620, 4)
    genre                                               plot  temp  topp
0  action  PAUL FLUSH: When the medical examiner must rul...   1.0   0.0
1  action  A last-ditch attempt to create an take-down sa...   1.0   0.0
2  action  This isn't a very successful film. It's a lot ...   1.0   0.0
3  action  A Florida man is found dead and his daughter w...   1.0   0.0
4  action  Juri reveals to Elvira that he has a problem w...   1.0   0.0


## Calculate similarity

### Jaccard similarity

In [0]:
def jaccard_similarity(str1, str2):
    """Jacard similarity, or, how overfit is our data?..."""
    s1 = set(str1.split(' '))
    s2 = set(str2.split(' '))
    return len(s1.intersection(s2)) / len(s1.union(s2))

In [0]:
def similarity_matrix(column, similarity_func, overwrite=False):
    """Calculate the pairwise similarity between texts of two lists."""
    # It's important to preserve the order of columns and index to compare multiple matrices later
    fpath = CACHE_DIR/(column + '.npz')
    if fpath.exists() and not overwrite:
        print("Loading from cache...")
        # Do not forget to uncompress
        return np.load(fpath)['sim_matrix']
    pbar = tqdm(total=len(gen_df.index)*len(train_df.index))
    # Fill similarity matrix
    # We can compress our matrix to float16:
    # Since the values are always in [0, 1] with maximum step of 1/256 (for jaccard_similarity)
    sim_matrix = np.empty((len(gen_df.index), len(train_df.index)), dtype=np.float16)
    sim_matrix[:, :] = np.nan
    for i, t1 in enumerate(gen_df[column]):
        for j, t2 in enumerate(train_df[column]):
            if t1 not in (None, '') and t2 not in (None, ''):
                sim_matrix[i, j] = similarity_func(t1, t2)
            pbar.update()
    pbar.close()
    # Reduce file size by applying NumPy compression
    np.savez_compressed(fpath, sim_matrix=sim_matrix)
    return sim_matrix

In [0]:
# Plot similarity
plot_jm = similarity_matrix('plot', jaccard_similarity)

print(plot_jm.shape)

 67%|██████▋   | 126283130/189852660 [1:00:00<44:51, 23618.29it/s]

### Universal sentence encoder

In [0]:
# Load inference model
model = hub.load(MODULE_URL)

In [0]:
def compute_embeddings(texts, fname, overwrite=False):
    """Compute a representation for each text."""
    fpath = CACHE_DIR/(fname + '.npz')
    if fpath.exists() and not overwrite:
        print("Loading from cache...")
        return np.load(fpath)['embeddings']
    embeddings = []
    for i in tqdm(range(0, len(texts), MAX_BATCH_SIZE)):
        embeddings.append(np.array(model(texts[i:i+MAX_BATCH_SIZE])))
    # np.inner has problems with np.float16
    embeddings = np.vstack(embeddings).astype(np.float32)
    assert(embeddings.shape[0] == len(texts))
    np.savez_compressed(fpath, embeddings=embeddings)
    return embeddings

In [0]:
gen_plot_embeddings = compute_embeddings(gen_df['plot'], 'gen_plot_embeddings')

print(gen_plot_embeddings.shape)

In [0]:
train_plot_embeddings = compute_embeddings(train_df['plot'], 'train_plot_embeddings')

print(train_plot_embeddings.shape)

In [0]:
%%time
# Compute pairwise similarity
plot_em = np.inner(gen_plot_embeddings, train_plot_embeddings)

print(plot_em.shape)

### Stack matrices

In [0]:
# Stack matrices together
jm = np.stack([plot_jm, plot_em], axis=2)

print(jm.shape)

In [0]:

# Release RAM
del plot_jm
del gen_plot_embeddings
del train_plot_embeddings
del plot_em

In [0]:
# Check if some generated titles/plots are empty or NaN
gen_df['plot'].apply(lambda x: x not in (None, '')).all()

## Evaluate similarity

In [0]:
# Correlation of similarities on different stages
print(pd.DataFrame({
    'plot_jaccard': jm[:, :, 0].flatten(), 
    'plot_embed': jm[:, :, 1].flatten()
}).corr())

### Maximum similarity per generated example

In [0]:
# For each generated sample, take the highest similarity found in the training dataset
maxsim_df = pd.DataFrame(jm.max(axis=1), columns=['plot_jaccard', 'plot_embed'], index=gen_df['plot'].tolist())

print(maxsim_df.head())

In [0]:
# The average similarity of generated plots to the real ones
maxsim_df.mean()

In [0]:
# The number of exact matches
(maxsim_df == 1).mean()

In [0]:
# Plot distribution of maximum similarity
maxsim_df.plot.hist(bins=30)

In [0]:
print(maxsim_df['plot_jaccard'].sort_values())

In [0]:
print(maxsim_df['plot_embed'].sort_values())

### Grouped by genre

In [0]:
GENRES = list(sorted(gen_df['genre'].unique()))

print(GENRES)

In [0]:
# Extract genre masks
gen_genre_masks = {}
for genre in GENRES:
    gen_genre_mask = gen_df.index.isin(gen_df[gen_df['genre'] == genre].index)
    gen_genre_masks[genre] = gen_genre_mask
train_genre_masks = {}
for genre in GENRES:
    train_genre_mask = train_df.index.isin(train_df[train_df['genres'].apply(lambda x: genre in x)].index)
    train_genre_masks[genre] = train_genre_mask

In [0]:
# How many examples in each genre
print(pd.DataFrame([(train_genre_masks[genre].sum(), gen_genre_masks[genre].sum()) 
for genre in GENRES], columns=['train', 'gen'], index=GENRES).sort_values(by='train', ascending=False))

In [0]:
def compute_genre_simmatrix(jm, gen_genre_masks, train_genre_masks, another_gen_mask=None):
    """Calculate mean similarity for each pair of genres."""
    genre_sim = np.empty((len(GENRES), len(GENRES), jm.shape[2]))
    pbar = tqdm(total=len(GENRES)**2)
    for i, gen_genre in enumerate(GENRES):
        for j, train_genre in enumerate(GENRES):
            gen_mask = gen_genre_masks[gen_genre]
            train_mask = train_genre_masks[train_genre]
            if another_gen_mask is not None:
                gen_mask = gen_mask & another_gen_mask
            genre_sim[i, j] = jm[gen_mask, :, :][:, train_mask, :].mean(axis=0).mean(axis=0)
            pbar.update()
    pbar.close()
    return genre_sim

In [0]:
genre_sim = compute_genre_simmatrix(jm, gen_genre_masks, train_genre_masks)

In [0]:
def plot_genre_heatmap(genre_sim):
    """Plot the genre similarity matrix as a heatmap."""
    genre_sim_df = pd.DataFrame(genre_sim, columns=GENRES, index=GENRES)
    sorted_genres = genre_sim_df.mean().sort_values().index.tolist()
    plt.figure(figsize=(10, 8))
    sns.heatmap(genre_sim_df.loc[sorted_genres, sorted_genres], cmap="YlGnBu")
    plt.show()

In [0]:
# Based on Jaccard similarity
# Generated plots are on the left, training plots on the bottom
# Dark horizontal lines mean the genre couldn't be learned sufficiently (few data?)
plot_genre_heatmap(genre_sim[:, :, 0])

In [0]:
# Average similarity between plots of the same genre
# Hints at expressiveness of each genre 
pd.Series(genre_sim[:, :, 0].diagonal(), index=GENRES).sort_values(ascending=False)

In [0]:
# Based on semantic similarity
# Dark points hint on semantics correlation of both genres
plot_genre_heatmap(genre_sim[:, :, 1])

In [0]:
pd.Series(genre_sim[:, :, 1].diagonal(), index=GENRES).sort_values(ascending=False)

### Grouped by metrics

In [0]:
# For each temperature, calculate the avg similarity of plots of the same genre
genre_temp_sims = {}
for temp, group in gen_df.groupby(by='temp'):
    gen_mask = gen_df.index.isin(group.index).tolist()
    genre_temp_sim = compute_genre_simmatrix(jm, gen_genre_masks, train_genre_masks, another_gen_mask=gen_mask)
    genre_temp_sims[temp] = [genre_temp_sim[:, :, sim].diagonal().mean() for sim in range(genre_temp_sim.shape[2])]
pd.DataFrame(genre_temp_sims, index=['plot_jaccard', 'plot_embed']).transpose().plot.bar()

In [0]:
# For each top_p, calculate the avg similarity of plots of the same genre
genre_topp_sims = {}
for topp, group in gen_df.groupby(by='topp'):
    gen_mask = gen_df.index.isin(group.index).tolist()
    genre_topp_sim = compute_genre_simmatrix(jm, gen_genre_masks, train_genre_masks, another_gen_mask=gen_mask)
    genre_topp_sims[topp] = [genre_topp_sim[:, :, sim].diagonal().mean() for sim in range(genre_topp_sim.shape[2])]
pd.DataFrame(genre_topp_sims, index=['plot_jaccard', 'plot_embed']).transpose().plot.bar()

### The most similar example

In [0]:
# Display the most similar example according to universal embeddings
gen_i, train_i = np.unravel_index(np.argmax(jm[:, :, 1]), jm.shape[:2])

In [0]:
jm[gen_i, train_i, :]

In [0]:
print(gen_df.iloc[gen_i])
print(gen_df.iloc[gen_i]['plot'])

In [0]:
print(train_df.iloc[train_i])
print(train_df.iloc[train_i]['plot'])

### The least similar examples

In [0]:
# Sort generated texts by the mean of all their similarities
sorted_i = np.argsort(np.max(np.mean(jm, axis=2), axis=1))

In [0]:
print('\n'.join(gen_df.iloc[sorted_i[:5]]['plot'].tolist()))

In [0]:
print(gen_df.iloc[sorted_i[0]])
print(gen_df.iloc[sorted_i[0]]['plot'])